# 03. Gold layer EDA (v2): ключевые слова, персоны, гео и «кто что делает»

Этот ноутбук анализирует **gold**-слой (Parquet) проекта *Media Intelligence Hub*:
- качество данных и диапазон времени;
- динамика ключевых слов;
- топ персон и гео;
- **действия персон**: какие глаголы чаще всего встречаются в предложениях, где упомянута персона.

Примечание: извлечение «действий» сделано эвристически (без синтаксического парсинга), поэтому это хорошая
базовая метрика для повестки, но не идеальный «кто-что-сделал». Ниже есть идеи, как усилить качество.


In [ ]:
from __future__ import annotations

from pathlib import Path
import re
from collections import Counter, defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# optional: для извлечения глаголов
try:
    import pymorphy2
    _MORPH_OK = True
    morph = pymorphy2.MorphAnalyzer()
except Exception as e:
    _MORPH_OK = False
    morph = None
    print("[WARN] pymorphy2 недоступен, блок 'действия' будет пропущен:", e)

PROJECT_ROOT = Path.cwd()


## 1) Загрузка данных

По умолчанию берём **последние** gold-файлы:
- RSS: `data/gold/articles_*_processed.parquet` (без `telegram`)
- Telegram: `data/gold/articles_*_telegram_*_processed.parquet`

Если хочешь анализировать конкретный файл — просто задай `RSS_GOLD` / `TG_GOLD` вручную.


In [ ]:
def latest_by_glob(pattern: str) -> Path:
    matches = sorted(PROJECT_ROOT.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    if not matches:
        raise FileNotFoundError(f"No files matched: {pattern}")
    return matches[0]

# Авто-выбор последних файлов
RSS_GOLD = None  # Path("data/gold/<your_file>.parquet")
TG_GOLD  = None  # Path("data/gold/<your_file>.parquet")

if RSS_GOLD is None:
    # RSS: исключаем телеграмные файлы по имени
    rss_candidates = [p for p in PROJECT_ROOT.glob("data/gold/articles_*_processed.parquet") if "telegram" not in p.name]
    if not rss_candidates:
        print("[WARN] Не нашли RSS gold (data/gold/articles_*_processed.parquet без telegram).")
        RSS_GOLD = None
    else:
        RSS_GOLD = sorted(rss_candidates, key=lambda p: p.stat().st_mtime, reverse=True)[0]

if TG_GOLD is None:
    tg_candidates = list(PROJECT_ROOT.glob("data/gold/articles_*_telegram_*_processed.parquet"))
    if not tg_candidates:
        print("[WARN] Не нашли Telegram gold (data/gold/articles_*_telegram_*_processed.parquet).")
        TG_GOLD = None
    else:
        TG_GOLD = sorted(tg_candidates, key=lambda p: p.stat().st_mtime, reverse=True)[0]

RSS_GOLD, TG_GOLD


In [ ]:
def load_gold(path: Path, source_label: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    df = df.copy()
    if "source_group" not in df.columns:
        df["source_group"] = source_label
    return df

frames = []
if RSS_GOLD is not None:
    frames.append(load_gold(RSS_GOLD, "rss"))
if TG_GOLD is not None:
    frames.append(load_gold(TG_GOLD, "telegram"))

if not frames:
    raise RuntimeError("Не удалось загрузить ни одного gold-файла. Проверь data/gold/ и пути.")

df = pd.concat(frames, ignore_index=True)
print("rows:", len(df))
print("columns:", len(df.columns))
df.head(3)


## 2) Санити-чек: время, дубликаты id, пропуски

Важно: дашборды «не обновляются» чаще всего потому, что **в ClickHouse пока нет новых публикаций**
(например, Telegram/ RSS собирались за 25 декабря). Здесь проверяем диапазон времени прямо по gold.


In [ ]:
# published_at может быть tz-naive (как в gold_to_clickhouse), приводим безопасно
df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")

print("published_at min:", df["published_at"].min())
print("published_at max:", df["published_at"].max())

if "id" in df.columns:
    s = df["id"].astype(str).str.strip()
    print("empty id:", int((s=="").sum()))
    print("unique id:", s.nunique())
    print("dupe ids:", int(s.duplicated().sum()))

# краткая сводка по источникам
if "source" in df.columns:
    display(df.groupby(["source_group", "source"]).size().sort_values(ascending=False).head(20))
else:
    display(df.groupby(["source_group"]).size())


## 3) Ключевые слова: топ и динамика

Предполагаем, что `keywords` в gold — строка с `;` (как в твоих витринах).


In [ ]:
def explode_keywords(d: pd.DataFrame) -> pd.DataFrame:
    if "keywords" not in d.columns:
        return pd.DataFrame(columns=["published_at", "source_group", "keyword"])
    k = d[["published_at", "source_group", "keywords"]].copy()
    k["keywords"] = k["keywords"].fillna("").astype(str)
    k = k.assign(keyword=k["keywords"].str.split(";")).explode("keyword")
    k["keyword"] = k["keyword"].fillna("").astype(str).str.strip().str.lower()
    k = k[k["keyword"] != ""]
    return k[["published_at", "source_group", "keyword"]]

kw = explode_keywords(df)
kw.head(5), len(kw)


In [ ]:
top_kw = kw["keyword"].value_counts().head(30)
top_kw


In [ ]:
# Динамика по топ-10 ключевым словам
TOP_N = 10
top_set = set(top_kw.head(TOP_N).index.tolist())
kw_top = kw[kw["keyword"].isin(top_set)].copy()

# округление до часа (можно заменить на 'min' / 'D')
kw_top["ts"] = kw_top["published_at"].dt.floor("H")

ts = (
    kw_top.groupby(["ts", "keyword"])
    .size()
    .reset_index(name="cnt")
    .pivot(index="ts", columns="keyword", values="cnt")
    .fillna(0)
    .sort_index()
)

ax = ts.plot(kind="line", figsize=(12, 5))
ax.set_title("Keyword trends (top-10)")
ax.set_xlabel("time")
ax.set_ylabel("count")
plt.show()


## 4) Персоны и гео: топ

Колонки ожидаются в формате `a;b;c` (как в gold).


In [ ]:
def explode_semicolon_col(d: pd.DataFrame, col: str, out_name: str) -> pd.DataFrame:
    if col not in d.columns:
        return pd.DataFrame(columns=["published_at", "source_group", out_name])
    x = d[["published_at", "source_group", col]].copy()
    x[col] = x[col].fillna("").astype(str)
    x = x.assign(**{out_name: x[col].str.split(";")}).explode(out_name)
    x[out_name] = x[out_name].fillna("").astype(str).str.strip().str.lower()
    x = x[x[out_name] != ""]
    return x[["published_at", "source_group", out_name]]

persons = explode_semicolon_col(df, "persons", "person")
geo = explode_semicolon_col(df, "geo", "geo_item")

persons["person"].value_counts().head(30)


In [ ]:
geo["geo_item"].value_counts().head(30)


## 5) Действия персон (эвристика)

Идея:
1) берём текст (приоритет `nlp_text` → `clean_text` → `raw_text`);
2) делим на предложения;
3) если в предложении упомянута персона — собираем **глаголы** этого предложения;
4) агрегируем по персоне.

Это не полноценный синтаксический разбор, но уже даёт хороший «срез повестки»:
- что чаще «делают» персоны в медиа;
- какие действия доминируют для конкретной персоны.


In [ ]:
if not _MORPH_OK:
    raise RuntimeError("pymorphy2 не установлен, блок 'действия' выполнить нельзя.")

_TOKEN_RE = re.compile(r"[A-Za-zА-Яа-яЁё-]+")
_SENT_SPLIT_RE = re.compile(r"(?<=[\.\!\?\…])\s+")

def _norm_text(s: str) -> str:
    s = (s or "")
    s = s.replace("ё", "е")
    return s

def extract_verbs(sentence: str) -> list[str]:
    sent = _norm_text(sentence).lower()
    verbs: list[str] = []
    for tok in _TOKEN_RE.findall(sent):
        p = morph.parse(tok)[0]
        if p.tag.POS in ("VERB", "INFN"):
            verbs.append(p.normal_form)
    return verbs

def split_sentences(text: str) -> list[str]:
    t = _norm_text(text).strip()
    if not t:
        return []
    # сначала заменим переносы на пробелы, чтобы не ломать предложения
    t = re.sub(r"\s+", " ", t)
    return [s.strip() for s in _SENT_SPLIT_RE.split(t) if s.strip()]

def iter_persons(persons_str: str) -> list[str]:
    if not persons_str:
        return []
    out = [p.strip().lower() for p in persons_str.split(";")]
    return [p for p in out if p]

def choose_text_row(row: pd.Series) -> str:
    for c in ("nlp_text", "clean_text", "raw_text"):
        if c in row and pd.notna(row[c]) and str(row[c]).strip():
            return str(row[c])
    return ""

def person_in_sentence(person: str, sentence: str) -> bool:
    # базовая проверка: подстрока
    sent = _norm_text(sentence).lower()
    p = _norm_text(person).lower()
    if p in sent:
        return True
    # fallback: для "владимир зеленский" проверим фамилию
    parts = p.split()
    if len(parts) >= 2:
        return parts[-1] in sent
    return False


In [ ]:
# собираем события "person -> verbs" построчно
rows = []
for _, r in df.iterrows():
    persons_str = str(r.get("persons") or "")
    plist = iter_persons(persons_str)
    if not plist:
        continue

    text = choose_text_row(r)
    if not text:
        continue

    for sent in split_sentences(text):
        verbs = extract_verbs(sent)
        if not verbs:
            continue

        for p in plist:
            if person_in_sentence(p, sent):
                for v in verbs:
                    rows.append((p, v))

actions = pd.DataFrame(rows, columns=["person", "verb"])
actions.head(), len(actions)


In [ ]:
# Топ глаголов в повестке (только предложения с упоминанием персон)
actions["verb"].value_counts().head(30)


In [ ]:
# Топ глаголов для топ-10 персон
top_persons = persons["person"].value_counts().head(10).index.tolist()
for p in top_persons:
    top_v = actions.loc[actions["person"] == p, "verb"].value_counts().head(10)
    print("\n", "="*60)
    print("PERSON:", p)
    print(top_v.to_string())


## 6) Идеи, как улучшить качество «кто что делает»

Если захочешь сделать метрику точнее (и «пристегнуть» её в пайплайн/ClickHouse), обычно идут по шагам:
1) **Ограничить окно**: брать глаголы не из всего предложения, а в окне ±N токенов вокруг упоминания персоны.
2) **Фильтр “шумных” глаголов**: сделать stop-list для «быть/стать/мочь/сказать/сообщить» и т.п.
3) **Глаголы-события**: сгруппировать синонимы (например, “заявить/сообщить/сказать” → “заявить”).
4) **Синтаксический разбор** (самый качественный вариант): определить, является ли персона субъектом глагола.
   Для русского это обычно `spacy` (ru_core_news_lg) или `stanza`. Это тяжелее, но даёт “агент → действие”.

Если ты скажешь, какой уровень качества нужен (эвристика / окно / синтаксис), я помогу оформить это как модуль
`src/processing/actions.py` и добавить в gold как `actions_persons` (например, JSON: {"персона": ["глагол1", ...]}).
